# Prepare / build Parquet Dataset

- step 1 was : data collection -> populate a sqlite db (basic deduplicate while populating data)
- -> **step 2** : cleaning / formating, trunk premium articles using Polars -> join articles & comments (tidy format) into parquet dataset

In [1]:
from pathlib import Path
import sqlite3
import json
import polars as pl
import pandas as pd

from datetime import datetime
import ast

In [2]:
pl.Config.set_tbl_rows(8)

polars.config.Config

### Load data (sqlite db)

In [3]:
# using Polars
db_path = "ukr.db"
connection_uri = "sqlite://" + db_path
query_articles = "SELECT * FROM articles"
query_comments = "SELECT * FROM comments"

In [4]:
df_articles = pl.read_database(query_articles, connection_uri, engine="connectorx")

In [5]:
df_articles.shape

(2825, 10)

In [6]:
#%%timeit
df_comments = pl.read_database(query_comments, connection_uri, engine="connectorx")

In [7]:
df_comments.shape

(244899, 3)

Curious about speed, using pandas, to load the biggest table "comments" (300k rows)

In [8]:
#%%timeit
with sqlite3.connect('ukr.db') as conn:
    pd_df_comments = pd.read_sql_query("SELECT * FROM comments", conn)

In [9]:
pd_df_comments.head(2)

,article_id,author,comment
0,3269346,Touch,Que va- t- on leur expliquer lorsque Mme Lepen...
1,3269346,Imprécateur.,"Bien pour le fond. Mais pour la forme, à t'on..."


In [10]:
with sqlite3.connect('ukr.db') as conn:
    pd_df_articles = pd.read_sql_query("SELECT * FROM articles", conn)
pd_df_articles.head(2)

,article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium
0,47863,https://www.lemonde.fr/europe/article/2015/07/...,La Russie entend bloquer la création d’un tri...,"Vol MH17 : un an après sa destruction, cinq p...","L’Australie, la Belgique, la Malaisie, les Pay...",2015-07-14 07:57:27.000000,"['international', 'economie', 'europe', 'ukrai...",Factuel,1,1
1,142250,https://www.lemonde.fr/europe/article/2016/09/...,L’enquête affirme que le missile provenait de...,Vol MH17 : la Russie clairement désignée,"Le missile Buk, qui a abattu le Boeing 777 de ...",2016-09-29 04:39:20.000000,"['international', 'europe', 'ukraine', 'pays-b...",Factuel,1,1


### Prepare data

**Notes** <br>
> - Using Polars, because why not. Comments do no need to be prepared (only 3 cols, good format), Articles does
> - Premium articles "content" will be cut to first 2500 characters, to --kind of, respect copyright (+- a non suscriber can see)
> - Change cols types (1 -> True, str(list) -> list etc. ; we had no choice when ingesting into our sqlite db. Cool thing is that we removed duplicates alrdy
> - While properly searching for ukraine conflict dates, we still collected older, non pertinent articles, remove these.

In [11]:
df_comments.head(2)

article_id,author,comment
i64,str,str
3269346,"""Touch ""","""Que va- t- on …"
3269346,"""Imprécateur.""","""Bien pour le f…"


In [12]:
df_comments.head(2)

article_id,author,comment
i64,str,str
3269346,"""Touch ""","""Que va- t- on …"
3269346,"""Imprécateur.""","""Bien pour le f…"


#### Format columns

In [13]:
bool_cols = [
    "allow_comments",
    "premium"
]
cat_cols = [
    "article_type"
]
time_cols = [
    "date"
]
str_list_cols = [
    "keywords"
]

In [14]:
# custom functions to be passed
def list_to_string(col) -> pl.Expr:
    return ast.literal_eval(col)

def int_to_bool(col) -> pl.Expr:
    return True if int(col) else False

In [15]:
# convert to bool, convert to real list, parse - format date, convert to categorical
df_articles = df_articles.with_columns([
    pl.col(bool_cols).apply(int_to_bool),
    pl.col(str_list_cols).apply(list_to_string),
    pl.col(time_cols).apply(lambda x: x.replace(".000000", "")).str.strptime(pl.Date, fmt="%Y-%m-%d %H:%M:%S"),
    pl.col(cat_cols).cast(pl.Categorical)
])

In [16]:
df_articles.head(3)

article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium
i64,str,str,str,str,date,list[str],cat,bool,bool
47863,"""https://www.le…","""La Russie ente…","""Vol MH17 : un …","""L’Australie, l…",2015-07-14,"[""international"", ""economie"", … ""russie""]","""Factuel""",true,true
142250,"""https://www.le…","""L’enquête aff…","""Vol MH17 : la …","""Le missile Buk…",2016-09-29,"[""international"", ""europe"", … ""russie""]","""Factuel""",true,true
142507,"""https://www.le…","""vidéo Abattu …","""Vol MH17 : la …","""Le 17 juillet …",2016-09-29,"[""international"", ""videos"", … ""russie""]","""Éclairage""",true,false


#### Check null values

In [17]:
for col in df_articles.get_columns():
    print(f'{col.name} - {col.is_null().sum()}')

article_id - 0
url - 0
title - 167
desc - 167
content - 0
date - 0
keywords - 0
article_type - 0
allow_comments - 0
premium - 0


In [18]:
# Let's investigate why null values -,-
df_articles.filter(pl.col("title").is_null())

article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium
i64,str,str,str,str,date,list[str],cat,bool,bool
3261362,"""https://www.le…",null,null,"""Nos contenus s…",2022-02-19,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
3261394,"""https://www.le…",null,null,"""Nos contenus s…",2022-02-20,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
3261596,"""https://www.le…",null,null,"""Pour approfond…",2022-02-21,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
3261619,"""https://www.le…",null,null,"""Pour approfond…",2022-02-22,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
…,…,…,…,…,…,…,…,…,…
3312114,"""https://www.le…",null,null,"""Retrouvez notr…",2023-02-25,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
3312149,"""https://www.le…",null,null,"""Retrouvez notr…",2023-02-26,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
3312202,"""https://www.le…",null,null,"""Retrouvez notr…",2023-02-27,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false
3312368,"""https://www.le…",null,null,"""Retrouvez notr…",2023-02-28,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Live Actu""",false,false


In [19]:
# those are the articles of type "Live actu", that we did not want to parse in the first place but that got through, remove
df_articles = df_articles.drop_nulls()

In [20]:
df_articles.shape

(2658, 10)

#### Remove articles that are older than desired period (start conflict 24/02/2022 + 1 year)

In [21]:
df_articles = df_articles.filter(pl.col("date") >= datetime(2022,2,24))
print(df_articles.shape, df_articles.head(2))

(2590, 10) shape: (2, 10)
┌──────────┬───────────┬───────────┬───────────┬───┬───────────┬────────────┬────────────┬─────────┐
│ article_ ┆ url       ┆ title     ┆ desc      ┆ … ┆ keywords  ┆ article_ty ┆ allow_comm ┆ premium │
│ id       ┆ ---       ┆ ---       ┆ ---       ┆   ┆ ---       ┆ pe         ┆ ents       ┆ ---     │
│ ---      ┆ str       ┆ str       ┆ str       ┆   ┆ list[str] ┆ ---        ┆ ---        ┆ bool    │
│ i64      ┆           ┆           ┆           ┆   ┆           ┆ cat        ┆ bool       ┆         │
╞══════════╪═══════════╪═══════════╪═══════════╪═══╪═══════════╪════════════╪════════════╪═════════╡
│ 3259703  ┆ https://w ┆ Le        ┆ Au        ┆ … ┆ ["interna ┆ Factuel    ┆ true       ┆ false   │
│          ┆ ww.lemond ┆ conflit   ┆ Festival  ┆   ┆ tional",  ┆            ┆            ┆         │
│          ┆ e.fr/actu ┆ russo-ukr ┆ de journa ┆   ┆ "europe", ┆            ┆            ┆         │
│          ┆ alite…    ┆ ainien,   ┆ lisme de  ┆   ┆ … "f…     ┆ 

#### Truncate premium articles to first 2500 characters

In [22]:
a = "f"
print(f"this is a {a} string")

this is a f string


In [23]:
print(f"premium articles: {len(df_articles.filter(pl.col('premium') == True))}/{len(df_articles)}")

premium articles: 1936/2590


In [24]:
df_articles_copyright = df_articles.with_columns([
    pl.when(pl.col("premium") == True).then(pl.col("content").apply(lambda x: x[0:2500])).otherwise(pl.col("content"))
])

In [25]:
df_articles_copyright.head(5)

article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium
i64,str,str,str,str,date,list[str],cat,bool,bool
3259703,"""https://www.le…","""Le conflit rus…","""Au Festival de…","""Parce qu’elle …",2022-07-16,"[""international"", ""europe"", … ""festival-international-de-journalisme-de-couthures""]","""Factuel""",true,false
3261407,"""https://www.le…","""Selon l’ONG, «…","""Guerre en Ukra…","""C’est l’un des…",2022-03-09,"[""international"", ""europe"", … ""crise-ukrainienne""]","""Factuel""",true,false
3261429,"""https://www.le…","""Dans un ouvrag…","""« Guerres d’in…","""Livre. Plus le…",2022-02-24,"[""international"", ""geopolitique"", … ""idees-livres""]","""Critique""",true,true
3261869,"""https://www.le…","""Après l’offen…","""Guerre en Ukra…","""Les autorités…",2022-02-24,"[""international"", ""proche-orient"", … ""crise-ukrainienne""]","""Factuel""",true,true
3261889,"""https://www.le…","""Hors énergie,…","""Crise ukrainie…","""Les sanctions …",2022-02-25,"[""economie"", ""international"", … ""crise-ukrainienne""]","""Factuel""",true,true


In [26]:
# check if premium articles = 2500 char
print(len(df_articles_copyright.filter(pl.col("premium") == True).select(pl.col("content")).rows()[0][0]))
print(len(df_articles_copyright.filter(pl.col("premium") == False).select(pl.col("content")).rows()[0][0]))

2500
4582


### Join articles, comments into single dataframe

**Notes** <br>
> - Pros : tidy format, one observation by rows, one variable per column ; one single dataset to work with
> - Cons : size : several comments per articles, so we duplicate article content & desc for every comments


In [27]:
df_articles_copyright.shape

(2590, 10)

In [28]:
df_comments.shape

(244899, 3)

In [29]:
# note : we now get fewer comments, because we previously removed out-of-period articles, and null (lives) with according article_id
dataset = df_articles_copyright.join(df_comments, on="article_id", how="left")
dataset.shape

(236643, 12)

In [30]:
dataset.head(2)

article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium,author,comment
i64,str,str,str,str,date,list[str],cat,bool,bool,str,str
3259703,"""https://www.le…","""Le conflit rus…","""Au Festival de…","""Parce qu’elle …",2022-07-16,"[""international"", ""europe"", … ""festival-international-de-journalisme-de-couthures""]","""Factuel""",true,false,"""Ricardo Uztarr…","""La question qu…"
3259703,"""https://www.le…","""Le conflit rus…","""Au Festival de…","""Parce qu’elle …",2022-07-16,"[""international"", ""europe"", … ""festival-international-de-journalisme-de-couthures""]","""Factuel""",true,false,"""Ricardo Uztarr…","""Salandre : les…"


Sanity checkk if null values

In [39]:
for col in dataset.get_columns():
    print(f'{col.name} - {col.is_null().sum()}')

article_id - 0
url - 0
title - 0
desc - 0
content - 0
date - 0
keywords - 0
article_type - 0
allow_comments - 0
premium - 0
author - 0
comment - 0


### Save to compressed parquet

In [33]:
dataset.write_parquet("lmd_ukraine.parquet", compression="gzip")

Read it back using Polars (800ms)

In [34]:
%%timeit
# using Polars
new_dataset_pl = pl.read_parquet('lmd_ukraine.parquet')

819 ms ± 12.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [35]:
new_dataset_pl = pl.read_parquet('lmd_ukraine.parquet')
new_dataset_pl.head(2)

article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium,author,comment
i64,str,str,str,str,date,list[str],cat,bool,bool,str,str
3259703,"""https://www.le…","""Le conflit rus…","""Au Festival de…","""Parce qu’elle …",2022-07-16,"[""international"", ""europe"", … ""festival-international-de-journalisme-de-couthures""]","""Factuel""",true,false,"""Ricardo Uztarr…","""La question qu…"
3259703,"""https://www.le…","""Le conflit rus…","""Au Festival de…","""Parce qu’elle …",2022-07-16,"[""international"", ""europe"", … ""festival-international-de-journalisme-de-couthures""]","""Factuel""",true,false,"""Ricardo Uztarr…","""Salandre : les…"


Read it back using Pandas (1,8 sec)

In [36]:
%%timeit
# using Pandas
new_dataset_pd = pd.read_parquet('lmd_ukraine.parquet')

1.89 s ± 102 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [37]:
new_dataset_pd = pd.read_parquet('lmd_ukraine.parquet')
new_dataset_pd.head(2)

,article_id,url,title,desc,content,date,keywords,article_type,allow_comments,premium,author,comment
0,3259703,https://www.lemonde.fr/actualite-medias/articl...,"Le conflit russo-ukrainien, qui mobilise les m...",Au Festival de journalisme de Couthures : la g...,Parce qu’elle est revenue frapper à nos porte...,2022-07-16,"[international, europe, ukraine, crise-ukraini...",Factuel,True,False,Ricardo Uztarroz,La question qui vaille et qui n'est pas posée...
1,3259703,https://www.lemonde.fr/actualite-medias/articl...,"Le conflit russo-ukrainien, qui mobilise les m...",Au Festival de journalisme de Couthures : la g...,Parce qu’elle est revenue frapper à nos porte...,2022-07-16,"[international, europe, ukraine, crise-ukraini...",Factuel,True,False,Ricardo Uztarroz,Salandre : les documents dont vous faîtes ét...


In [38]:
new_dataset_pd.dtypes

article_id           int64
url                 object
title               object
desc                object
content             object
date                object
keywords            object
article_type      category
allow_comments        bool
premium               bool
author              object
comment             object
dtype: object